In [ ]:
import pygame
import random
import math
pygame.init()

class DrawInformation:
    BLACK = 0, 0, 0
    WHITE = 255, 255, 255
    GREEN = 0, 255, 0
    RED = 255, 0, 0
    BACKGROUND_COLOR = 20,5,56
    PINK=(237,7,89)
    GRADIENTS = [
        (1, 255, 205),
        (30, 200, 192),
        (47, 190, 184)
    ]

    FONT = pygame.font.SysFont('Segoe UI', 30)
    LARGE_FONT = pygame.font.SysFont('Segoe UI', 40)

    SIDE_PAD = 100  #padding including both left and right
    TOP_PAD = 150

    def __init__(self, width, height, lst):
        self.width = width
        self.height = height

        self.window = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Sorting Algorithm Visualization")
        self.set_list(lst)

    def set_list(self, lst):
        self.lst = lst
        self.min_val = min(lst)
        self.max_val = max(lst)

        self.block_width = round((self.width - self.SIDE_PAD) / len(lst))
        self.block_height = math.floor((self.height - self.TOP_PAD) / (self.max_val - self.min_val))  # floor otherwise bar over the top padding and not able to clean the screen
        self.start_x = self.SIDE_PAD // 2                                                                            


def draw(draw_info, algo_name, ascending):      # drawing window
    draw_info.window.fill(draw_info.BACKGROUND_COLOR)

    title = draw_info.LARGE_FONT.render(f"{algo_name} - {'Ascending' if ascending else 'Descending'}", 1, draw_info.GREEN)
    draw_info.window.blit(title, (draw_info.width/2 - title.get_width()/2 , 5))

    #Font. render() to create an image (Surface) of the text, then blit this image onto another Surface.
    controls = draw_info.FONT.render("R - Reset      SPACE - Start Sorting      A - Ascending      D - Descending", 1, draw_info.WHITE)
    
    # . blit() is how you copy the contents of one Surface to another
    draw_info.window.blit(controls, (draw_info.width/2 - controls.get_width()/2 , 55))  # 44.4   (controls,(x,y))

    sorting = draw_info.FONT.render("I - Insertion Sort     B - Bubble Sort", 1, draw_info.WHITE)
    draw_info.window.blit(sorting, (draw_info.width/2 - sorting.get_width()/2 , 95))

    draw_list(draw_info)
    pygame.display.update()     


def draw_list(draw_info, color_positions={}, clear_bg=False):    # drawing list on screen
    lst = draw_info.lst      # gives the list of random numbers

    if clear_bg:    # no need to draw static instruct while sorting so onlyclear the ractangle having the list drawn 
        clear_rect = (draw_info.SIDE_PAD//2, draw_info.TOP_PAD, 
                        draw_info.width - draw_info.SIDE_PAD, draw_info.height - draw_info.TOP_PAD) #(x,y,width,height)->rectangle
        pygame.draw.rect(draw_info.window, draw_info.BACKGROUND_COLOR, clear_rect)       #-->drawing rectangle in place of list

    for i, val in enumerate(lst):     #i,val ---> index,values at that index
        x = draw_info.start_x + i * draw_info.block_width
        y = draw_info.height - (val - draw_info.min_val) * draw_info.block_height   # 30.11

        color = draw_info.GRADIENTS[i % 3]

        if i in color_positions:
            color = color_positions[i] 

        pygame.draw.rect(draw_info.window, color, (x, y, draw_info.block_width, draw_info.height))

    if clear_bg:    #->imp to update the rectange after drawing rec while sorting
        pygame.display.update()


def generate_starting_list(n, min_val, max_val):  # generate the list having random values in range [min_val, max_val]
    lst = []

    for _ in range(n):
        val = random.randint(min_val, max_val)    # generating values range [int_min,int_max]
        lst.append(val)

    return lst


def bubble_sort(draw_info, ascending=True):
    lst = draw_info.lst

    for i in range(len(lst) - 1):
        for j in range(len(lst) - 1 - i):
            num1 = lst[j]
            num2 = lst[j + 1]

            if (num1 > num2 and ascending) or (num1 < num2 and not ascending):
                lst[j], lst[j + 1] = lst[j + 1], lst[j]
                draw_list(draw_info, {j: draw_info.PINK, j + 1: draw_info.RED}, True)  #(,)
                yield True  #yield in Python can be used like the return statement in a function. When done so, 
                #the function instead of returning the output, it returns a generator that can be iterated upon.
                #You can then iterate through the generator to extract items.
    return lst

def insertion_sort(draw_info, ascending=True):
    lst = draw_info.lst

    for i in range(1, len(lst)):
        current = lst[i]

        while True:
            ascending_sort = i > 0 and lst[i - 1] > current and ascending
            descending_sort = i > 0 and lst[i - 1] < current and not ascending

            if not ascending_sort and not descending_sort:
                break

            lst[i] = lst[i - 1]
            i = i - 1
            lst[i] = current
            draw_list(draw_info, {i - 1: draw_info.PINK, i: draw_info.RED}, True)
            yield True

    return lst


def main():
    run = True
    clock = pygame.time.Clock()

    n = 50
    min_val = 0
    max_val = 100

    lst = generate_starting_list(n, min_val, max_val)    # generate the list having random values in range [min_val, max_val]
    draw_info = DrawInformation(1000, 600, lst)      # initializing class DrawInformation with def __init__(self, width, height, lst):
    sorting = False
    ascending = True

    sorting_algorithm = bubble_sort
    sorting_algo_name = "Bubble Sort O(N^2)"
    sorting_algorithm_generator = None

    while run:                           # pygame event loop constantly running in the background otherwise if we something
                                        # on screen then the function will immediatly ends
        clock.tick(60)   # no. of times the loop run per second  --->helps to speed up the sorting 

        if sorting:
            try:
                next(sorting_algorithm_generator)
            except StopIteration:
                sorting = False
        else:
            draw(draw_info, sorting_algo_name, ascending)  # draw fxn used to fill background with white and update and ...

        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # if we use only eventy then we can close window by clicking on red cross 
                run = False

            if event.type != pygame.KEYDOWN:
                continue

            if event.key == pygame.K_r:                     ## on pressing key r
                lst = generate_starting_list(n, min_val, max_val)    # generate a list
                draw_info.set_list(lst)                              # then draw the list 
                sorting = False   #-------> we have to ensure to set "sorting = False "  when ever key_r is pressed
            elif event.key == pygame.K_SPACE and sorting == False:  # sorting == False to ensure that we donot start sorting if we
                                                                    # are already sorting
                sorting = True
                sorting_algorithm_generator = sorting_algorithm(draw_info, ascending)
            elif event.key == pygame.K_a and not sorting:         # we are setting ascending on pressing key_a when we are not sorting
                ascending = True
            elif event.key == pygame.K_d and not sorting:      # we are setting ascending on pressing key_awhen we are not sorting
                ascending = False
            elif event.key == pygame.K_i and not sorting:
                sorting_algorithm = insertion_sort
                sorting_algo_name = "Insertion Sort O(N^2)"
            elif event.key == pygame.K_b and not sorting:
                sorting_algorithm = bubble_sort
                sorting_algo_name = "Bubble Sort O(N^2)"



    pygame.quit()


if __name__ == "__main__":
    main()